In [54]:
import pandas as pd

# 도매 소매처코드 기준 명단 만들기

In [ ]:
import pandas as pd

# 엑셀 파일의 시트명 목록 보기
excel_file = pd.ExcelFile("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx")
print(excel_file.sheet_names)
df_1 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", sheet_name=excel_file.sheet_names[0], skiprows=1).iloc[:, 1:]

['伊藤忠食品', 'コゲツ', '加藤産業', '国分', '三井物産', '三菱食品', '藤徳物産', 'カナカン', 'ヤマキ', '旭食品', '湧川']


In [18]:
df_1.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_1

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,152884,大阪,1300735,1300063,伊藤忠食品株式会社西日本営業部,6154405,N3000035,田口大士
1,145292,大阪,1300735,1300064,伊藤忠食品株式会社中四国支店,6014803,N3000032,三澤暢太
2,303020,大阪,1300735,1300064,伊藤忠食品株式会社中四国支店,6022411,N3000032,三澤暢太
3,309800,大阪,1300735,1302960,伊藤忠食品株式会社西日本営業本部（CVS）,6070901,N3000030,赤平雄祐
4,300802,大阪,1300735,1302960,伊藤忠食品株式会社西日本営業本部（CVS）,6070901,N3000030,赤平雄祐
...,...,...,...,...,...,...,...,...
70,307921,東海,1300735,1300062,伊藤忠食品株式会社東海営業本部,6039190,N3000038,藤本俊逸
71,307921,東海,1300735,1300062,伊藤忠食品株式会社東海営業本部,6039190,N3000038,藤本俊逸
72,243514,東京,1300732,1302959,伊藤忠食品株式会社東日本営業本部（CVS）,6033130,N3000013,その他
73,239037,東京,1300732,1300061,伊藤忠食品株式会社東日本営業部,6164501,N3000023,矢野昭夫


In [21]:
df_2 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", sheet_name=excel_file.sheet_names[1], skiprows=1).iloc[:, 1:]
df_2.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_2.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,1001,北九州支店,1300771,1300190,コゲツ産業株式会社北九州支店,6041622,N3000039,八塚琴音
1,1025,北九州支店,1300771,1300190,コゲツ産業株式会社北九州支店,6041611,N3000039,八塚琴音
2,1822,北九州支店,1300771,1300190,コゲツ産業株式会社北九州支店,6033130,N3000013,その他
3,1238,北九州支店,1300771,1300190,コゲツ産業株式会社北九州支店,6019771,N3000039,八塚琴音
4,1323,北九州支店,1300771,1300190,コゲツ産業株式会社北九州支店,6041639,N3000039,八塚琴音


In [22]:
df_3 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[2], skiprows=1).iloc[:, 1:]
df_3.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_3.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,35521,福岡,1300712,1300006,加藤産業株式会社九州支社福岡支店,6003728,N3000019,田内慶之
1,32423,福岡,1300712,1300006,加藤産業株式会社九州支社福岡支店,6003851,N3000019,田内慶之
2,3242300001,福岡,1300712,1300006,加藤産業株式会社九州支社福岡支店,6003851,N3000019,田内慶之
3,204,福岡,1300712,1300006,加藤産業株式会社九州支社福岡支店,6003674,N3000019,田内慶之
4,34824,福岡,1300712,1300006,加藤産業株式会社九州支社福岡支店,6003674,N3000019,田内慶之


In [23]:
df_4 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[3], skiprows=1).iloc[:, 1:]
df_4.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_4.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,10012313,群馬支社,1300766,1300824,国分関信越株式会社群馬支社,6152401,N3000003,長田悟
1,10029807,群馬支社,1300766,1300824,国分関信越株式会社群馬支社,6000325,N3000003,長田悟
2,10050462,群馬支社,1300766,1300824,国分関信越株式会社群馬支社,6025949,N3000003,長田悟
3,10051344,群馬支社,1300766,1300824,国分関信越株式会社群馬支社,6152505,N3000003,長田悟
4,13029702,群馬支社,1300766,1300824,国分関信越株式会社群馬支社,6047928,N3000034,佐瀬雄一


In [24]:
df_5 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[4], skiprows=1).iloc[:, 1:]
df_5.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_5.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,110179,首都圏支社 中央営業部営業第四室 関信越,1300726,1301657,三井物産流通Ｇ（株）三井食品ユニット首都圏中央営業4室関信越,6052872,N3000003,長田悟
1,100029,首都圏 首都圏西一室,1300726,1305046,三井物産流通グループ株式会社首都圏支社首都圏西営業部,6051365,N3000041,澤野亮
2,100110,首都圏 首都圏西一室,1300726,1305046,三井物産流通グループ株式会社首都圏支社首都圏西営業部,6022462,N3000003,長田悟
3,112935,首都圏 首都圏西一室,1300726,1305046,三井物産流通グループ株式会社首都圏支社首都圏西営業部,6017899,N3000003,長田悟
4,116804,首都圏 首都圏西一室,1300726,1305046,三井物産流通グループ株式会社首都圏支社首都圏西営業部,6051370,N3000041,澤野亮


In [25]:
df_6 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[5], skiprows=1).iloc[:, 1:]
df_6.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_6.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,1145861,612,1300740,1301412,三菱食品株式会社東北支社加食営業第二部,6152523,N3000016,田和章一
1,1264847,612,1300740,1301412,三菱食品株式会社東北支社加食営業第二部,6039269,N3000016,田和章一
2,1264847,612,1300740,1301412,三菱食品株式会社東北支社加食営業第二部,6039269,N3000016,田和章一
3,1160931,403,1300740,1303675,三菱食品株式会社ＣＶＳ本部北海道,6014888,N3000016,田和章一
4,1160931,403,1300740,1303675,三菱食品株式会社ＣＶＳ本部北海道,6014888,N3000016,田和章一


In [26]:
df_7 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[6], skiprows=1).iloc[:, 1:]
df_7.columns = ["소매처코드", "지역", "PY코드", "SP코드", "대표슈퍼명", "대표슈퍼코드", "담당자ID", "담당자명"]
df_7.head()

,소매처코드,지역,PY코드,SP코드,대표슈퍼명,대표슈퍼코드,담당자ID,담당자명
0,412796,岡山支店,1301341,1300032,藤徳物産株式会社岡山支店,6163102,N3000032,三澤暢太
1,444508,岡山支店,1301341,1300032,藤徳物産株式会社岡山支店,6152527,N3000032,三澤暢太
2,444599,岡山支店,1301341,1300032,藤徳物産株式会社岡山支店,6035552,N3000032,三澤暢太
3,750648,岡山支店,1301341,1300032,藤徳物産株式会社岡山支店,6152413,N3000032,三澤暢太
4,007040,岡山支店,1301341,1300032,藤徳物産株式会社岡山支店,6152664,N3000032,三澤暢太


In [36]:
df_8 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[7], skiprows=1).iloc[:, 1:]
df_8.columns = ["소매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
df_8.head()

,소매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,株式会社どんたく,6062768,■株式会社どんたく,N3000038,藤本俊逸
1,有限会社スーパーしんや,6154075,■有限会社スーパーしんや,N3000034,佐瀬雄一
2,アルビス株式会社,6152411,■アルビス株式会社,N3000038,藤本俊逸
3,株ＭＡＮＰＯＷ ＦＯＯＤＳ Ｓ,6152498,■株式会社ＭＡＮＰＯＷ ＦＯＯＤＳ,N3000038,藤本俊逸
4,スギＨＤ株式会社（スギ薬局）,6022452,■株式会社スギ薬局,N3000011,佐々木俊文


In [37]:
df_9 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[8], skiprows=1).iloc[:, 1:]
df_9.columns = ["소매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
df_9.head()

,소매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,有限会社カネデン近藤商店,■その他,6033130,N3000013,その他
1,有限会社静岡ワールドフーズ,■有限会社静岡ワールドフーズ,6152694,N3000011,佐々木俊文
2,マルキフード,■マルキフード,6153994,N3000011,佐々木俊文
3,ふじや,■株式会社富士屋,6022467,N3000011,佐々木俊文
4,フジ物産株式会社,■フジ物産株式会社,6152465,N3000011,佐々木俊文


In [38]:
df_10 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[9], skiprows=1).iloc[:, 1:]
df_10.columns = ["소매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
df_10.head()

,소매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,株式会社サンディ,6000594,■株式会社サンディ,N3000035,田口大士
1,山崎製パン株式会社,6001059,■山崎製パン株式会社,N3000004,鈴木啓介
2,三菱マテリアル直島生活協同組合,6001056,■三菱マテリアル直島生活協同組合,N3000032,三澤暢太
3,株式会社ダイコク,6000885,■株式会社ダイコク,N3000035,田口大士
4,株式会社瀬戸内,6001005,■株式会社瀬戸内,N3000032,三澤暢太


In [39]:
df_11 = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx", 
                     sheet_name=excel_file.sheet_names[10], skiprows=1).iloc[:, 1:]
df_11.columns = ["소매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
df_11.head()

,소매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,㈱リウボウストアー,6152584,■株式会社リウボウストア,N3000019,田内慶之
1,株式会社丸大,6152595,■株式会社丸大,N3000019,田内慶之
2,金秀商事株式会社,6152638,■金秀商事株式会社,N3000019,田内慶之
3,株式会社ＪＡおきなわＡコープ,6152416,■おきなわ農業協同組合,N3000019,田内慶之
4,(有)仲松商事,6154086,■有限会社仲松商事,N3000019,田内慶之


In [52]:
domae_retail_1 = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7])
domae_retail_2 = pd.concat([df_8, df_9, df_10, df_11])

domae_retail_1[["소매처코드", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]].to_csv("database/csv/domae_retail_1.csv", index=False)
domae_retail_2[["소매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]].to_csv("database/csv/domae_retail_2.csv", index=False)

# 판매처별 슈퍼 명단 만들기

In [55]:
# 최근 1년치 판매처-슈퍼 매핑
master_code = pd.read_excel("/Users/nongshim/Downloads/条件マスターコード一覧26.01.29.xlsx").drop_duplicates()
print(master_code.shape)
master_code.columns=["판매처코드", "판매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
master_code.to_csv("database/csv/dist_retail.csv", index=False)

(424, 6)


# 사용자별 슈퍼 명단 만들기

In [ ]:
# 최근 1년치 판매처-슈퍼 매핑
print(len(set(master_code["담당자ID"])))
master_code.head()

14


,판매처코드,판매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,1303017,株式会社日本アクセス広域リテール営業本部九州支店鹿児島営業所,6035526,株式会社南九州ファミリーマート,N3000039,八塚琴音
1,1302976,株式会社トーカン商品統括部（CVS),6003788,ファミリーマート,N3000034,佐瀬雄一
2,1303568,株式会社日本アクセス広域リテール営業本部加工食品飲料部,6003788,ファミリーマート,N3000034,佐瀬雄一
3,1303567,株式会社日本アクセス広域リテール営業本部東北営業課,6003788,ファミリーマート,N3000034,佐瀬雄一
4,1303569,株式会社日本アクセス広域リテール営業本部中部・北陸営業課,6003788,ファミリーマート,N3000034,佐瀬雄一


In [56]:
# 슈퍼 담당자 매핑
df = pd.read_excel("/Users/nongshim/Downloads/代表スーパーコード一覧.XLSX")
df = df.drop_duplicates()
print(df.shape)
print(len(set(df["INTRANET ID"])))
df.columns = ['대표슈퍼코드', '대표슈퍼명', '담당자ID', '담당자명', 'ID']
df.head() 
# o30240021

(1445, 5)
16


,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
0,6000015,■有限会社木村屋,N3000041,澤野亮,o30250003
1,6000104,■株式会社千葉薬品,N3000011,佐々木俊文,o30240005
2,6000131,■株式会社京成ストア,N3000041,澤野亮,o30250003
3,6000181,■株式会社くすりの福太郎,N3000011,佐々木俊文,o30240005
4,6000185,■Ａコープ,N3000011,佐々木俊文,o30240005


In [126]:
df[df["담당자ID"].isin(["N3000025", "N3000013"])]

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
453,6020609,■株式会社日立物流,N3000013,営業管理 営業社員,-
577,6033130,■その他,N3000013,営業管理 営業社員,-
768,6056563,■株式会社JR東日本リテールネット,N3000025,◆ＣＶＳ,-
1430,6196420,■辛メーター株式会社,N3000013,営業管理 営業社員,-


In [10]:
master_code[master_code["담당자ID"].isin(["N3000013"])].head()

,판매처코드,판매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
10,1300206,株式会社マルイチ産商食品事業部長野支店,6033130,あいにー食品館,N3000013,その他
20,1300206,株式会社マルイチ産商食品事業部長野支店,6033130,株式会社田村商店,N3000013,その他
30,1300142,株式会社トーカン,6033130,清水屋,N3000013,その他
47,1300002,株式会社日本アクセス北関東支店,6033130,カインズ,N3000013,その他
66,1304968,ヤマエ久野株式会社東京支店食品課,6033130,Firsto,N3000013,その他


In [57]:
df[df.대표슈퍼코드=="6153980"]

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID


In [58]:
master_code[master_code.대표슈퍼코드=="6153980"]

,판매처코드,판매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명


In [ ]:
# 최근 1년치 판매처-슈퍼 매핑 + 슈퍼 담당자 매핑 병합
# o30240021
col = ['대표슈퍼코드', '대표슈퍼명', '담당자ID', '담당자명']
super_user_map = pd.concat([df[col].drop_duplicates(), master_code[col].drop_duplicates()]).sort_values(by='담당자ID',ignore_index=True)
super_user_map["대표슈퍼명"] = super_user_map["대표슈퍼명"].apply(lambda x: x.replace("■", ""))
super_user_map["대표슈퍼명"] = super_user_map["대표슈퍼명"].apply(lambda x: x.replace("株式会社", ""))
super_user_map = pd.merge(super_user_map, df[["담당자ID","ID"]].drop_duplicates(), on='담당자ID', how='left')
print(super_user_map.shape)
super_user_map.head()

(1832, 5)


,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
0,6021019,信濃屋食品,N3000003,長田悟,o30240015
1,6049544,有限会社八百賢,N3000003,長田悟,o30240015
2,6049800,新印食品,N3000003,長田悟,o30240015
3,6035512,与野フードセンター,N3000003,長田悟,o30240015
4,6164500,マエダ(マークスグループ),N3000003,長田悟,o30240015


In [60]:
super_user_map[super_user_map["대표슈퍼코드"]=="6153980"]

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID


In [61]:
super_user_map.to_csv("database/csv/retail_user.csv", index=False)

# SAP 슈퍼 명단 만들기

In [67]:
sap_retail = pd.read_excel("/Users/nongshim/Downloads/SAP_retail.XLSX")
sap_retail.head()
sap_retail.columns = ["대표슈퍼코드", "대표슈퍼명","대표슈퍼명2", "판매처코드", "판매처명"]
sap_retail.drop(columns=["대표슈퍼명2"], inplace=True)
sap_retail.to_csv("database/csv/sap_retail.csv", index=False)

# 단가 명단 만들기

In [ ]:
unit_price = pd.read_excel("/Users/nongshim/Downloads/★단가리스트★26.02.10.xls", sheet_name="価格表2306", skiprows=2)
unit_price = unit_price[["商品コード", "商品名", "内容量\n換算値", "問屋　　仕切", "本部長　　ｃｔ", "JANコード　      　　　    (ﾊﾗﾙ→UPC)"]]
unit_price.columns = ["제품코드", "제품명", "제품용량", "시키리", "본부장", "JANCD"]
unit_price.head()

,제품코드,제품명,제품용량,시키리,본부장,JANCD
0,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694
1,101003583,辛ラーメンキムチ カップ,68.0,140.0,88.0,8801043038096
2,101004421,辛ラーメン激辛カップ,68.0,140.0,88.0,8801043052429
3,101002078,本場韓国コムタンラーメン,75.0,140.0,88.0,8801043011549
4,101005016,ノグリラーメンカップ(HOT),62.0,140.0,88.0,8801043059602


In [174]:
unit_price.to_csv("database/csv/unit_price.csv", index=False)

# 유사도 매핑

In [1]:
# 슈퍼명 유사도 테스트 (CSV 참고용). 실제 앱(내 담당만 등)은 DB pg_trgm 사용 → 결과가 다를 수 있음.
# 여기서는 difflib으로 유사도 0~1 계산 (앱과 동일한 알고리즘 아님)
import pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import re

def split_name_and_capacity(name):
    """
    "제품명" 등에서 용량(예: １２０Ｇ, 120g, 90ML 등)를 분리해서 (이름, 용량) 튜플로 반환.
    용량이 없으면 (원본, None) 반환.
    단, 단위가 그람(g/ｇ/G/Ｇ)일 때는 숫자만 추출해서 반환.
    """
    # 전각/반각/소문자 등 다양한 표기(숫자+단위) (g, G, ｇ, ML 등)
    # 아래 패턴은 마지막에 등장하는 용량(예: １２０Ｇ, 120g, 90ML, 12コ) 등 추출
    # 'Ｇ'도 포함하도록 단위 집합 확장, 전각 및 반각 대소문자 모두 포함!
    pattern = re.compile(
        r"([０-９0-9]+\.?[０-９0-9]*)([gGｇＧmMｍＭlLリットルﾘｯﾄﾙ個コ袋])\s*$"
    )
    # 전각 숫자를 반각으로 바꾸기 위한 테이블
    to_hankaku = str.maketrans("０１２３４５６７８９", "0123456789")
    name = name.strip()
    m = pattern.search(name)
    if m:
        num = m.group(1).translate(to_hankaku)
        unit = m.group(2)
        # 그람 단위(g, G, ｇ, Ｇ)이면 숫자만 남기고, 아니면 단위 붙여 반환
        if unit in ["g", "G", "ｇ", "Ｇ"]:
            cap = num
        else:
            cap = f"{num}{unit}"
        base = name[: m.start()].strip()
        return base, cap
    return name, None


def similarity(a: str, b: str) -> float:
    """두 문자열 유사도 0~1 (pg_trgm과 비슷한 개념)."""
    a, b = (a or "").strip(), (b or "").strip()
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

def find_similar_supers(
    query: str,
    csv_path: str,
    col: str,
    top_k: int = 10,
    min_similarity: float = 0.0,
    sub_col: str = None,
    sub_query: str = None,
    sub_min_similarity: float = 0.0,
):
    """
    입력한 슈퍼명과 retail_user.csv의 대표슈퍼명(또는 지정 컬럼) 중 유사도 높은 순으로 반환.
    2개 이상의 컬럼 유사도 필터 지원: col과 sub_col을 각각 비교.
    """
    path = Path(csv_path)
    if not path.exists():
        raise FileNotFoundError(f"파일 없음: {path}")
    df = pd.read_csv(path, encoding="utf-8")
    if col not in df.columns:
        raise KeyError(f"컬럼 없음: {col}. 사용 가능: {list(df.columns)}")
    if sub_col and sub_col not in df.columns:
        raise KeyError(f"서브 컬럼 없음: {sub_col}. 사용 가능: {list(df.columns)}")

    query = (query or "").strip()
    sub_query = (sub_query or "").strip() if sub_query is not None else None

    rows = []
    for _, row in df.iterrows():
        # 1차 컬럼(주 컬럼) 유사도 계산
        name = (row.get(col) or "")
        if isinstance(name, float) and pd.isna(name):
            name = ""
        name = str(name).strip()
        primary_score = similarity(query, name)
        # 2차 컬럼(옵션) 유사도 계산
        if sub_col and sub_query is not None:
            sub_value = (row.get(sub_col) or "")
            if isinstance(sub_value, float) and pd.isna(sub_value):
                sub_value = ""
            sub_value = str(sub_value).strip()
            sub_score = similarity(sub_query, sub_value)
        else:
            sub_score = None

        # 1차 필터링 (주 컬럼)
        if primary_score >= min_similarity:
            # 2차 필터링 (서브 컬럼, 없으면 패스)
            if (sub_col and sub_query is not None):
                if sub_score >= sub_min_similarity:
                    row_dict = {**row.to_dict(), f"{col}_similarity": round(primary_score, 4), f"{sub_col}_similarity": round(sub_score, 4)}
                    rows.append(row_dict)
            else:
                row_dict = {**row.to_dict(), "similarity": round(primary_score, 4)}
                rows.append(row_dict)
    # 정렬 기준: 1차는 주 컬럼, 둘 다 있으면 (주, 보조 동시 정렬)
    if sub_col and sub_query is not None:
        rows.sort(key=lambda x: (x[f"{col}_similarity"], x[f"{sub_col}_similarity"]), reverse=True)
    else:
        rows.sort(key=lambda x: x.get("similarity", 0), reverse=True)
    return pd.DataFrame(rows).head(top_k)

In [2]:
INPUT_SUPER_NAME = "岩田産業(株)     本社"   
INPUT_SUPER_NAME = "	全国農業（協組連） エーコープ九州"
top = find_similar_supers(INPUT_SUPER_NAME, csv_path="database/csv/retail_user.csv", col="대표슈퍼명", top_k=5, min_similarity=0.2)
print(f"입력: '{INPUT_SUPER_NAME}' → 유사도 상위 {len(top)}건")
display(top)

입력: '	全国農業（協組連） エーコープ九州' → 유사도 상위 5건


,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID,similarity
0,6125210,全国農業協同組合連合会,N3000011,佐々木俊文,o30240005,0.5000
1,6041618,Ａコープ九州,N3000039,八塚琴音,o30240021,0.4348
2,6047988,生活協同組合ユーコープ,N3000034,佐瀬雄一,o30240012,0.4286
3,6001077,全国農業協同組合連合会高知県本部,N3000032,三澤暢太,o30240006,0.4242
4,6033934,エーコープ宮城,N3000016,田和章一,o30240014,0.4167


In [190]:
INPUT_SUPER_NAME = "農心ジャパン 辛カップラーメン ６８Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/csv/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心ジャパン 辛カップラーメン', 용량: '68'
입력: '農心ジャパン 辛カップラーメン' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.6957,0.6667
1,101003068,★ハラル辛カップラーメン,68.0,148.0,108.0,★031146270606,0.5926,0.6667
2,101002007,ユッケジャンカップ,86.0,91.0,78.0,8801043010139,0.5000,0.3333
3,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.4545,0.3333
4,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.4545,0.2857


In [187]:
INPUT_SUPER_NAME = "農心 辛ラーメン 袋 １２０Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/csv/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心 辛ラーメン 袋', 용량: '120'
입력: '農心 辛ラーメン 袋' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101000461,辛ラーメン1P,120.0,126.0,78.0,8801043150606,0.5882,0.7500
1,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.5882,0.2857
2,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.5882,0.2500
3,101001977,辛ラーメン5P,600.0,630.0,345.0,8801043009720,0.5882,0.2500
4,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.5556,0.2857
